<a href="https://colab.research.google.com/github/55Ankur55/Rgroup/blob/master/Intent_Classififcation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In order for torch to use the GPU, we need to identify and specify the GPU as the device. Later, in our training loop, we will load data onto the device. 

In [ ]:
import torch
 
if torch.cuda.is_available():    

       
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
 
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
!pip install transformers

     |████████████████████████████████| 890kB 5.4MB/s 
     |████████████████████████████████| 890kB 30.9MB/s 
     |████████████████████████████████| 1.1MB 40.8MB/s 
     |████████████████████████████████| 3.0MB 53.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=549854caf0fd15c57a4b93df8d68950470906abb71edc0728c3c1221f64604a1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!gdown --id 1etHkU6Fr9R9-O9QeAXOtTT_VVWYo3HaF --output train.csv
# !gdown --id 1OlcvGWReJMuyYQuOZm149vHWwPtlboR6 --output train.csv
# !gdown --id 1Oi5cRlTybuIF2Fl5Bfsr-KkqrXrdt77w --output valid.csv
# !gdown --id 1ep9H6-HvhB4utJRLVcLzieWNUSG3P_uF --output test.csv

Downloading...
From: https://drive.google.com/uc?id=1etHkU6Fr9R9-O9QeAXOtTT_VVWYo3HaF
To: /content/train.csv
100% 13.1k/13.1k [00:00<00:00, 4.94MB/s]


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train=pd.read_csv("train.csv")
# valid=pd.read_csv("valid.csv")
# test=pd.read_csv("test.csv")
train.head(40)

,text,intent
0,i want to take a from delhi at 838 am and ar...,TrainAvailable
1,trains available from delhi to mumbai on thurs...,TrainAvailable
2,show trains from mumbai to delhi on thursday,TrainAvailable
3,all trains from delhi to mumbai,TrainAvailable
4,trains from delhi to mumbai,TrainAvailable
5,all trains from delhi to mumbai leaving after ...,TrainAvailable
6,cheapest train fare from delhi to mumbai,TrainFare
7,round trip fares from delhi to mumbai under 10...,TrainFare
8,first AC fares from delhi to mumbai,TrainFare
9,I would like to have some information on a tic...,TrainFare


In [ ]:
# train = train.append(valid).reset_index(drop=True)
# train.shape
# train.head()

In [ ]:
sentences = train.text.values
labels = train.intent.values

In [ ]:
sentences[2]

' show  trains from mumbai to delhi on thursday'

# 3. Tokenization & Input Formatting

In this section, we'll transform our dataset into the format that BERT can be trained on.

## 3.1. BERT Tokenizer


To feed our text to BERT, it must be split into tokens, and then these tokens must be mapped to their index in the tokenizer vocabulary.

The tokenization must be performed by the tokenizer included with BERT--the below cell will download this for us. We'll be using the "uncased" version here.


In [ ]:
from transformers import BertTokenizer

print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [ ]:
 print(' Original: ', sentences[0])

print('Tokenized: ', tokenizer.tokenize(sentences[0]))

print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

 Original:   i want to take a  from delhi at 838 am and arrive in mumbai at 1110 in the morning
Tokenized:  ['i', 'want', 'to', 'take', 'a', 'from', 'delhi', 'at', '83', '##8', 'am', 'and', 'arrive', 'in', 'mumbai', 'at', '111', '##0', 'in', 'the', 'morning']
Token IDs:  [1045, 2215, 2000, 2202, 1037, 2013, 6768, 2012, 6640, 2620, 2572, 1998, 7180, 1999, 8955, 2012, 11118, 2692, 1999, 1996, 2851]


In [ ]:
max_len = 0

 
for sent in sentences:
 
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
 
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  23


In [ ]:
input_labels = []
 
k = 1;
mp = {}
mpp= {}
for sent in labels:
    mp[sent]=0;
for sent in labels:
    if(mp[sent] == 0):
        print(sent)
        mp[sent] = k
        mpp[k]=sent
        k = k + 1
    input_labels.append(mp[sent]-1)
print('Original: ', labels[0])
print('Token IDs:', input_labels[0])
print('Original: ', labels[1])
print('Token IDs:', input_labels[1])
print('Original: ', labels[2])
print('Token IDs:', input_labels[2])
print('Original: ', labels[3])
print('Token IDs:', input_labels[3])
print(k)

TrainAvailable
TrainFare
GetDistance
BookTickets
Trainlocation
TrainRoute
Original:  TrainAvailable
Token IDs: 0
Original:  TrainAvailable
Token IDs: 0
Original:  TrainAvailable
Token IDs: 0
Original:  TrainAvailable
Token IDs: 0
7


In [ ]:
input_ids = []
attention_masks = []
 
for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True,  
                        max_length = 64,            
                        pad_to_max_length = True,
                        return_attention_mask = True,    
                        return_tensors = 'pt',      
                        truncation = True,
                   )
    
   
    input_ids.append(encoded_dict['input_ids'])
    
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(input_labels)

print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:   i want to take a  from delhi at 838 am and arrive in mumbai at 1110 in the morning
Token IDs: tensor([  101,  1045,  2215,  2000,  2202,  1037,  2013,  6768,  2012,  6640,
         2620,  2572,  1998,  7180,  1999,  8955,  2012, 11118,  2692,  1999,
         1996,  2851,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


# Split dataset in Train and Validation

In [ ]:
from torch.utils.data import TensorDataset, random_split

In [ ]:
dataset = TensorDataset(input_ids, attention_masks, labels)
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

  216 training samples
   24 validation samples


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler


In [ ]:
batch_size = 32

train_dataloader = DataLoader(
            train_dataset,   
            sampler = RandomSampler(train_dataset),  
            batch_size = batch_size  
        )
 
validation_dataloader = DataLoader(
            val_dataset,  
            sampler = SequentialSampler(val_dataset),  
            batch_size = batch_size 
        )

# Sequence Classification of Intents

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",  
    num_labels = k-1,    
    output_attentions = False,  
    output_hidden_states = False,
)

model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,  
                  eps = 1e-8  
                )


In [ ]:
from transformers import get_linear_schedule_with_warmup
 
epochs = 4
 
total_steps = len(train_dataloader) * epochs
 
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,  
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np
 
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    ''' 
    elapsed_rounded = int(round((elapsed)))
     
    return str(datetime.timedelta(seconds=elapsed_rounded))


# Training Model

In [ ]:
import random
import numpy as np
  
seed_val = 42
 
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
 
training_stats = []
  
total_t0 = time.time()
  
for epoch_i in range(0, epochs):
    
   
 
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
  
    t0 = time.time()
  
    total_train_loss = 0
 
    model.train()
  
    for step, batch in enumerate(train_dataloader):
  
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
 
        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
 
        model.zero_grad()        
 
       
        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
 
     
        total_train_loss += loss.item()
 
        loss.backward()
 
       
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
 
        optimizer.step()
 
        scheduler.step()
 
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    training_time = format_time(time.time() - t0)
 
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
   
 
    print("")
    print("Running Validation...")
 
    t0 = time.time()
 
    model.eval()
 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
 
    for batch in validation_dataloader:
        
     
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        
        with torch.no_grad():        
 
          
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
         
        total_eval_loss += loss.item()
 
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
 
       
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        
 
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
 
    avg_val_loss = total_eval_loss / len(validation_dataloader)
  
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))
 
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )
 
print("")
print("Training complete!")
 
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...

  Average training loss: 1.73
  Training epcoh took: 0:00:01

Running Validation...
  Accuracy: 0.62
  Validation Loss: 1.48
  Validation took: 0:00:00

======== Epoch 2 / 4 ========
Training...

  Average training loss: 1.45
  Training epcoh took: 0:00:01

Running Validation...
  Accuracy: 0.71
  Validation Loss: 1.37
  Validation took: 0:00:00

======== Epoch 3 / 4 ========
Training...

  Average training loss: 1.30
  Training epcoh took: 0:00:01

Running Validation...
  Accuracy: 0.75
  Validation Loss: 1.27
  Validation took: 0:00:00

======== Epoch 4 / 4 ========
Training...

  Average training loss: 1.23
  Training epcoh took: 0:00:01

Running Validation...
  Accuracy: 0.83
  Validation Loss: 1.22
  Validation took: 0:00:00

Training complete!
Total training took 0:00:06 (h:mm:ss)


# Prediction

In [ ]:
sentences = [
  "available trains from delhi to mumbai",
  "what is the route of sln express",
]
 

In [ ]:
input_ids = []
attention_masks = []

for sent in sentences:
   
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, 
                        max_length = 64,          
                        pad_to_max_length = True,
                        return_attention_mask = True,  
                        return_tensors = 'pt', 
                        truncation=True,    
                   )
    
    input_ids.append(encoded_dict['input_ids'])
    
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
 

batch_size = 32  

prediction_data = TensorDataset(input_ids, attention_masks )
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

model.eval()

predictions , true_labels = [], []

for batch in prediction_dataloader:
 
  batch = tuple(t.to(device) for t in batch)
  
  b_input_ids, b_input_mask  = batch
  
  with torch.no_grad():
      
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  logits = logits.detach().cpu().numpy()
  
  predictions.append(logits)


print('    DONE.')

Predicting labels for 2 test sentences...
    DONE.


In [ ]:
flat_predictions = np.concatenate(predictions, axis=0)
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

 
print(mpp[flat_predictions[0]+1])
print(mpp[flat_predictions[1]+1])
 

TrainAvailable
TrainRoute


In [ ]:
import os

output_dir = './model_save/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
print("Saving model to %s" % output_dir)


model_to_save = model.module if hasattr(model, 'module') else model  
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


Saving model to ./model_save/


('./model_save/vocab.txt',
 './model_save/special_tokens_map.json',
 './model_save/added_tokens.json')

In [ ]:
!ls -l --block-size=K ./model_save/

total 428012K
-rw-r--r-- 1 root root      1K Sep 17 17:14 config.json
-rw-r--r-- 1 root root 427769K Sep 17 17:14 pytorch_model.bin
-rw-r--r-- 1 root root      1K Sep 17 17:14 special_tokens_map.json
-rw-r--r-- 1 root root      1K Sep 17 17:14 tokenizer_config.json
-rw-r--r-- 1 root root    227K Sep 17 17:14 vocab.txt


In [ ]:
!ls -l --block-size=M ./model_save/pytorch_model.bin

-rw-r--r-- 1 root root 418M Sep 17 17:14 ./model_save/pytorch_model.bin


In [ ]:
ls

model_save/  sample_data/  train.csv


In [ ]:
model = BertForSequenceClassification.from_pretrained(output_dir)
tokenizer = BertTokenizer.from_pretrained(output_dir)

model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
model_save_name = 'irctc.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
torch.save(model.state_dict(), path)

In [ ]:
model_save_name = 'irctc.pt'
path = F"/content/gdrive/My Drive/{model_save_name}"
model.load_state_dict(torch.load(path))

<All keys matched successfully>